In [1]:
import sys
!{sys.executable} -m pip install pandas

In [3]:
import numpy as np
import pandas as pd
import os
import copy
from tvb.simulator.lab import models, connectivity, coupling, integrators, monitors
from tvb.simulator.simulator import Simulator

/opt/anaconda3/envs/tvb_env/lib/python3.10/site-packages/tvb/datatypes/surfaces.py:60: UserWarning: Geodesic distance module is unavailable; some functionality for surfaces will be unavailable.
  warnings.warn(msg)


In [5]:
np.random.seed(50)

In [7]:
n_subjects = 50
results = []
output_dir = "healthy_groupsim_results"
os.makedirs(output_dir, exist_ok=True)

In [9]:
for i in range(n_subjects):
    print(f"Running subject {i+1}...")

    wc = models.WilsonCowan(
        c_ee=np.array([np.random.normal(20.0, 1.0)]),
        c_ei=np.array([np.random.normal(12.0, 0.5)]),
        c_ie=np.array([np.random.normal(18.0, 1.0)]),
        c_ii=np.array([np.random.normal(3.0, 0.3)]),
        P=np.array([np.random.normal(0.45, 0.02)]),
        tau_e=np.array([10.0]),
        tau_i=np.array([25.0]),
    )

    conn = connectivity.Connectivity.from_file("connectivity_76.zip")
    conn.configure()
    if conn.region_labels is None or len(conn.region_labels) == 0:
        conn.region_labels = np.array([f"Region_{j}" for j in range(conn.number_of_regions)])

    sim = Simulator(
        model=wc,
        connectivity=conn,
        coupling=coupling.Scaling(a=np.array([0.003])),
        integrator=integrators.HeunDeterministic(dt=0.1),
        monitors=(monitors.Raw(variables_of_interest=np.array([0]), period=1.0),)
    )
    sim.simulation_length = 15000
    sim.configure()

    try:
        output = sim.run()[0]
        raw_data = output[1]  
        print("Raw data shape before squeeze:", raw_data.shape)

        raw_data = np.squeeze(raw_data)
        print("Raw data shape after squeeze:", raw_data.shape)

        if raw_data.ndim == 2 and raw_data.shape[1] == conn.number_of_regions:
            ts = raw_data[:, 18]  
            alpha_power = np.mean(np.abs(np.fft.rfft(ts))**2)

            fc_matrix = np.corrcoef(raw_data.T)
            np.save(os.path.join(output_dir, f"fc_matrix_{i}.npy"), fc_matrix)
            mean_fc = np.mean(fc_matrix[np.triu_indices(fc_matrix.shape[0], k=1)])
        else:
            print(f"Unexpected shape for subject {i+1}. Skipping.")
            alpha_power = np.nan
            mean_fc = np.nan

    except Exception as e:
        print(f"Error for subject {i+1}: {e}")
        alpha_power = np.nan
        mean_fc = np.nan

    results.append({
        "subject_id": f"healthy_{i+1}",
        "alpha_power_dlpfc": alpha_power,
        "mean_fc_strength": mean_fc
    })

Running subject 1...
2025-05-17 11:50:19,704 - ERROR - tvb.basic.readers - Could not import tvb_data Python module for default data-set!
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tvb_env/lib/python3.10/site-packages/tvb/basic/readers.py", line 222, in try_get_absolute_path
    module_import = importlib.import_module(relative_module)
  File "/opt/anaconda3/envs/tvb_env/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1004, i

In [11]:
df = pd.DataFrame(results)
df.to_csv(os.path.join(output_dir, "healthy_groupsim_metrics.csv"), index=False)
print("All simulations completed. Results saved to:", output_dir)

All simulations completed. Results saved to: healthy_groupsim_results
